#### Creating a connection to SQL Server using SQLAlchemy and ODBC, to allow loading JSON tables into the database.

In [1]:
!pip install sqlalchemy


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 11.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Nikita\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import json
import pyodbc

In [3]:
# SQL Server connection settings
server = 'NIKITA-PC'
database = 'CyberNewsBot'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}&Trusted_Connection=yes"
)
engine = create_engine(connection_string)


### 🔹 Step 1: Loading Posted News json file

#####     1.Load the posted_news_ud.json file.
#####     2.Convert the keywords column to a string.
#####     3.Check if an article already exists by text_hash.
#####     4.Load only the latest news into the PostedNews table.

In [4]:
# Reading from JSON file
posted_df = pd.read_json(r"posted_news_ud.json")

# Convert keywords to a comma-separated string list
if 'keywords' in posted_df.columns:
    posted_df['keywords'] = posted_df['keywords'].apply(
        lambda x: ','.join(x) if isinstance(x, list) else str(x)
    )

#  Filter duplicates by text_hash
existing_hashes = pd.read_sql("SELECT text_hash FROM PostedNews", con=engine)
new_posted_df = posted_df[~posted_df['text_hash'].isin(set(existing_hashes['text_hash']))]

# Loading into the database
if not new_posted_df.empty:
    new_posted_df.to_sql("PostedNews", con=engine, if_exists="append", index=False, chunksize=1000)
    print(f"✅ Loaded {len(new_posted_df)} new articles to PostedNews.")
else:
    print("ℹ️ No new articles to load into PostedNews.")


✅ Loaded 90 new articles to PostedNews.


### 🔹 Step 2: Loading SkippedNews json file

#####     1.Load the dictionary from skipped_news_ud.json.
#####     2.Check if an article already exists by text_hash
#####     3.Load only the latest news into the SkippedNews table


In [5]:
# Reading from JSON file
with open(r"skipped_news_ud.json", encoding='utf-8') as f:
    skipped_data = list(json.load(f).values())
skipped_df = pd.DataFrame(skipped_data)

#  Filter duplicates by text_hash
existing_skipped = pd.read_sql("SELECT text_hash FROM SkippedNews", con=engine)
new_skipped_df = skipped_df[~skipped_df['text_hash'].isin(set(existing_skipped['text_hash']))]
new_skipped_df = new_skipped_df.drop_duplicates(subset='text_hash')

# Loading into the database
if not new_skipped_df.empty:
    new_skipped_df.to_sql("SkippedNews", con=engine, if_exists="append", index=False, chunksize=1000)
    print(f"Loaded {len(new_skipped_df)} new articles to SkippedNews.")
else:
    print("No new articles to load into SkippedNews.")


Loaded 116 new articles to SkippedNews.


### Step 3: General tests
#Print some examples of the failed articles and perform a Missing Values ​​check to monitor data quality.

In [6]:
print("\n Sample of newly skipped articles:")
print(new_skipped_df[['title', 'reason']].head(3))

print("\n Missing value check (SkippedNews):")
print(new_skipped_df.isnull().sum())



 Sample of newly skipped articles:
                                                  title  \
1497  Markets/Coverages: Markel Launches Affirmative...   
1498  Marks and Spencer could face 12% drop in profi...   
1499  AI risks creating a cyber crunch for the UK's ...   

                                  reason  
1497           Duplicate by summary hash  
1498  Article text is empty or too short  
1499                  Duplicate by title  

 Missing value check (SkippedNews):
title             0
url               0
fail_count        0
date              0
reason            0
text_hash         0
summary           0
source            0
published_date    0
published_time    0
rss_source        0
dtype: int64
